In [1]:
import os
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader
import matplotlib.pyplot as plt

In [3]:
image_path = (r"C:\Users\rajan\Downloads\NewGen\Pan Card\train")

In [4]:
data = DataLoader.from_folder(image_path)
train_data, rest_data = data.split(0.8)

INFO:tensorflow:Load image with size: 696, num_label: 2, labels: Not Pan Card, Pan Card.


In [5]:
validation_data, test_data = rest_data.split(0.5)

In [6]:
model = image_classifier.create(train_data, model_spec=model_spec.get('efficientnet_lite0'), validation_data=validation_data, epochs = 20)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/20


C:\Users\rajan\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


17/17 [==============================] - 18s 934ms/step - loss: 0.3153 - accuracy: 0.9393 - val_loss: 0.2290 - val_accuracy: 1.0000
Epoch 2/20
17/17 [==============================] - 15s 906ms/step - loss: 0.2412 - accuracy: 0.9982 - val_loss: 0.2289 - val_accuracy: 1.0000
Epoch 3/20
17/17 [==============================] - 14s 818ms/step - loss: 0.2177 - accuracy: 1.0000 - val_loss: 0.2018 - val_accuracy: 1.0000
Epoch 4/20
17/17 [==============================] - 14s 805ms/step - loss: 0.2085 - accuracy: 1.0000 - val_loss: 0.2005 - val_accuracy: 1.0000
Epoch 5/20
17/17 [==============================] - 14s 795ms/step - loss: 0.2069 - accuracy: 1.0000 - val_loss: 0.2002 - val_accuracy: 1.0000
Epoch 6/20
17/17 [==============================] - 13s 783ms/step - loss: 0.2059 - accuracy: 1.0000 - val_loss: 0.2003 - val_accuracy: 1.0000
Epoch 7/20
17/17 [==============================] - 15s 854ms/step - loss: 0.2054 - accuracy: 1.0000 - val_loss: 0.1998 - val_accuracy: 1.0000
Epoch 8/20

In [7]:
loss, accuracy = model.evaluate(test_data)

3/3 [==============================] - 2s 429ms/step - loss: 0.1994 - accuracy: 1.0000


In [8]:
model.export(export_dir='.')

INFO:tensorflow:Assets written to: C:\Users\rajan\AppData\Local\Temp\tmpwucxyrya\assets


INFO:tensorflow:Assets written to: C:\Users\rajan\AppData\Local\Temp\tmpwucxyrya\assets
C:\Users\rajan\Anaconda3\lib\site-packages\tensorflow\lite\python\convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in C:\Users\rajan\AppData\Local\Temp\tmpwpiv1p03\labels.txt


INFO:tensorflow:Saving labels in C:\Users\rajan\AppData\Local\Temp\tmpwpiv1p03\labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: .\model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: .\model.tflite


TFLite

In [10]:
import tensorflow.lite as tflite
# Load TFLite model and allocate tensors.
interpreter = tflite.Interpreter(model_path=r"C:\Users\rajan\Downloads\model.tflite")
#allocate the tensors
interpreter.allocate_tensors()

In [11]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [12]:
import cv2
import numpy as np
image_path=(r"C:\Users\rajan\Downloads\NewGen\Pan Card\validation\Pan card\YODLY4129I.jpg")
img = cv2.imread(image_path)
img = cv2.resize(img,(224,224))
#Preprocess the image to required size and cast
input_shape = input_details[0]['shape']
input_tensor= np.array(np.expand_dims(img,0))

In [13]:
input_index = interpreter.get_input_details()[0]["index"]
interpreter.set_tensor(input_index, input_tensor)
interpreter.invoke()
output_details = interpreter.get_output_details()

In [14]:
output_data = interpreter.get_tensor(output_details[0]['index'])
pred = np.squeeze(output_data)

In [15]:
class_ind = {
0 :'Pan Card',
1:'Not Pan Card'}

In [16]:
highest_pred_loc = np.argmax(pred)

In [17]:
document_name = class_ind[highest_pred_loc]
print(f"This Document is {document_name}")

This Document is Not Pan Card
